# DataFrames: Reading in messy data
     
In the [01-data-access](./01-data-access.ipynb) example we show how Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  One key difference, when using Dask Dataframes is that instead of opening a single file with a function like [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), we typically open many files at once with [dask.dataframe.read_csv](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.read_csv). This enables us to treat a collection of files as a single dataset. Most of the time this works really well. But real data is messy and in this notebook we will explore a more advanced technique to bring messy datasets into a dask dataframe.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

<Client: 'inproc://10.1.0.145/7276/1' processes=1 threads=4, memory=2.00 GB>

## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [2]:
import dask
df = dask.datasets.timeseries()
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
2000-01-01      int64  object  float64  float64
2000-01-02        ...     ...      ...      ...
...               ...     ...      ...      ...
2000-01-30        ...     ...      ...      ...
2000-01-31        ...     ...      ...      ...
Dask Name: make-timeseries, 30 tasks

In [3]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name, index=False);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [4]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [5]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
                int64  object  float64  float64
                  ...     ...      ...      ...
...               ...     ...      ...      ...
                  ...     ...      ...      ...
                  ...     ...      ...      ...
Dask Name: read-csv, 30 tasks

In [6]:
df.head()

id      name         x         y
0   972     Laura  0.273973 -0.165516
1  1002   Michael -0.510155  0.105734
2  1001    Xavier -0.902875 -0.856783
3   978   Norbert  0.117538  0.070167
4  1020  Patricia  0.836880  0.837742

Let's look at some statistics on the data

In [7]:
df.describe().compute()

id             x             y
count  2.592000e+06  2.592000e+06  2.592000e+06
mean   1.000008e+03 -2.230901e-05  3.764726e-04
std    3.159632e+01  5.773526e-01  5.770455e-01
min    8.230000e+02 -9.999994e-01 -9.999990e-01
25%    9.790000e+02 -4.932313e-01 -4.925213e-01
50%    1.000000e+03  9.024390e-03  7.174105e-03
75%    1.021000e+03  5.076600e-01  5.057808e-01
max    1.157000e+03  9.999994e-01  9.999981e-01

# Make some messy data

Now this works great, and in most cases dd.read_csv or dd.read_parquet etc are the preferred way to read in large collections of data files into a dask dataframe, but real world data is often very messy and some files may be broken or badly formatted. To simulate this we are going to create some fake messy data by tweaking our example csv files. For the file `data/2000-01-05.csv` we will replace with no data and for the file `data/2000-01-07.csv` we will remove the `y` column 

In [8]:
# corrupt the data in data/2000-01-05.csv
with open('data/2000-01-05.csv', 'w') as f:
    f.write('')

In [9]:
# remove y column from data/2000-01-07.csv
import pandas as pd
df = pd.read_csv('data/2000-01-07.csv')
del df['y']
df.to_csv('data/2000-01-07.csv', index=False)

In [10]:
!head data/2000-01-05.csv

In [11]:
!head data/2000-01-07.csv

id,name,x
1010,Sarah,0.8046934484310249
972,Hannah,-0.4497275498475606
1050,Hannah,0.3625963996052908
979,Dan,-0.4604754795571235
966,George,-0.8876807248061698
1012,Quinn,0.7306406308811328
1000,Wendy,0.3064075365312089
997,Yvonne,0.9811749808407844
952,Norbert,-0.614069776601879


# Reading the messy data

Let's try reading in the collection of files again

In [12]:
df = dd.read_csv('data/2000-*-*.csv')

In [13]:
df.head()

id      name         x         y
0   972     Laura  0.273973 -0.165516
1  1002   Michael -0.510155  0.105734
2  1001    Xavier -0.902875 -0.856783
3   978   Norbert  0.117538  0.070167
4  1020  Patricia  0.836880  0.837742

Ok this looks like it worked, let us calculate the dataset statistics again

In [14]:
df.describe().compute()

distributed.worker - WARNING -  Compute Failed
Function:  execute_task
args:      ((subgraph_callable, (<function pandas_read_text at 0x7f6f001adee0>, <function read_csv at 0x7f6f03897670>, (<function read_block_from_file at 0x7f6f00140160>, <OpenFile '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-07.csv'>, 0, 64000000, b'\n'), b'id,name,x,y\n', {}, {'id': dtype('int64'), 'name': dtype('O'), 'x': dtype('float64'), 'y': dtype('float64')}, ['id', 'name', 'x', 'y'], False, False, None), 'read-csv-c6e9f489c3cb41d7f840704985dd949a'))
kwargs:    {}
Exception: ValueError('Length mismatch: Expected axis has 3 elements, new values have 4 elements')



ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements

So what happened? 

When creating a dask dataframe from a collection of files, dd.read_csv samples the first few files in the dataset to determine the datatypes and columns available. Since it has not opened all the files it does not now if some of them are corrupt. Hence, `df.head()` works since it is only looking at the first file. `df.describe.compute()` fails because of the corrupt data in `data/2000-01-05.csv`

# Building a delayed reader

To get around this problem we are going to use a more advanced technique to build our dask dataframe. This method can also be used any time some custom logic is required when reading each file. Essentially, we are going to build a function that uses  pandas and some error checking and returns a pandas dataframe. If we find a bad data file we will either find a way to fix/clean the data or we will return and empty pandas dataframe with the same structure as the good data.

In [15]:
import numpy as np
import io

def read_data(filename):
    
    # for this to work we need to explicitly set the datatypes of our pandas dataframe 
    dtypes = {'id': int, 'name': str, 'x': float, 'y': float}
    try:
        # try reading in the data with pandas 
        df = pd.read_csv(filename, dtype=dtypes)
    except:
        # if this fails create an empty pandas dataframe with the same dtypes as the good data
        df = pd.read_csv(io.StringIO(''), names=dtypes.keys(), dtype=dtypes)
    
    # for the case with the missing column, add a column of data with NaN's
    if 'y' not in df.columns:
        df['y'] = np.NaN
        
    return df

Let's test this function on a good file and the two bad files

In [16]:
# test function on a normal file
read_data('data/2000-01-01.csv').head()

id      name         x         y
0   972     Laura  0.273973 -0.165516
1  1002   Michael -0.510155  0.105734
2  1001    Xavier -0.902875 -0.856783
3   978   Norbert  0.117538  0.070167
4  1020  Patricia  0.836880  0.837742

In [17]:
# test function on the empty file
read_data('data/2000-01-05.csv').head()

Empty DataFrame
Columns: [id, name, x, y]
Index: []

In [18]:
# test function on the file missing the y column
read_data('data/2000-01-07.csv').head()

id    name         x   y
0  1010   Sarah  0.804693 NaN
1   972  Hannah -0.449728 NaN
2  1050  Hannah  0.362596 NaN
3   979     Dan -0.460475 NaN
4   966  George -0.887681 NaN

# Assembling the dask dataframe

First we take our `read_data` function and convert it to a dask delayed function

In [19]:
from dask import delayed
read_data = delayed(read_data)

Let us look at what the function does now

In [20]:
df = read_data('data/2000-01-01.csv')
df

Delayed('read_data-0bc8cac9-23ba-40f0-8c4d-cdb81d5ee4fb')

It creates a delayed object, to actually run read the file we need to run `.compute()`

In [21]:
df.compute()

id      name         x         y
0       972     Laura  0.273973 -0.165516
1      1002   Michael -0.510155  0.105734
2      1001    Xavier -0.902875 -0.856783
3       978   Norbert  0.117538  0.070167
4      1020  Patricia  0.836880  0.837742
...     ...       ...       ...       ...
86395   994     Wendy  0.683664  0.173054
86396  1074     Sarah  0.881958  0.391513
86397  1034    Ursula -0.609401  0.656726
86398   949   Norbert  0.402216 -0.395646
86399   997     Edith -0.366206  0.627728

[86400 rows x 4 columns]

Now let's build a list of all the available csv files

In [22]:
# loop over all the files
from glob import glob
files = glob('data/2000-*-*.csv')
files

['data/2000-01-27.csv',
 'data/2000-01-05.csv',
 'data/2000-01-18.csv',
 'data/2000-01-02.csv',
 'data/2000-01-12.csv',
 'data/2000-01-09.csv',
 'data/2000-01-19.csv',
 'data/2000-01-13.csv',
 'data/2000-01-21.csv',
 'data/2000-01-10.csv',
 'data/2000-01-20.csv',
 'data/2000-01-14.csv',
 'data/2000-01-25.csv',
 'data/2000-01-01.csv',
 'data/2000-01-11.csv',
 'data/2000-01-07.csv',
 'data/2000-01-03.csv',
 'data/2000-01-22.csv',
 'data/2000-01-15.csv',
 'data/2000-01-17.csv',
 'data/2000-01-04.csv',
 'data/2000-01-06.csv',
 'data/2000-01-28.csv',
 'data/2000-01-16.csv',
 'data/2000-01-23.csv',
 'data/2000-01-08.csv',
 'data/2000-01-30.csv',
 'data/2000-01-26.csv',
 'data/2000-01-24.csv',
 'data/2000-01-29.csv']

Now we run the delayed read_data function on each file in the list

In [23]:
df = [read_data(file) for file in files]
df

[Delayed('read_data-5e620f74-2994-4747-adcc-6c51ed6239c5'),
 Delayed('read_data-d074b40d-33dd-490b-8449-a21ed1f3ff9a'),
 Delayed('read_data-119c0328-064f-4bf0-92e0-5d49b9b131c4'),
 Delayed('read_data-d0ea01b5-67d2-4671-a4e4-fe348d318126'),
 Delayed('read_data-b51e01b8-151a-42b9-88af-d3732daafba1'),
 Delayed('read_data-2a23c731-c8c6-45ce-a2ff-3473b46320f3'),
 Delayed('read_data-1e839eb8-ebc8-4ffb-a5de-a576d0b71421'),
 Delayed('read_data-06c3cfec-e05e-4157-b20e-134db193b0c4'),
 Delayed('read_data-625b04f9-6a65-4eb0-828a-e5513d6b2b79'),
 Delayed('read_data-6bce75f4-2f0f-46ca-bfd6-aca9331e04b3'),
 Delayed('read_data-66eeb119-9d54-4920-9815-4fd1908fd621'),
 Delayed('read_data-769009a3-1061-4b84-85a3-a044e0e74192'),
 Delayed('read_data-76898852-8caf-4afd-8782-391fb03b7849'),
 Delayed('read_data-8a8a45c0-635f-4738-9e3e-e9081c8fe1e5'),
 Delayed('read_data-3992dfba-b0eb-4b29-a697-6a80db26e037'),
 Delayed('read_data-3fbc24f1-f11e-41bc-bdcb-ffa35bde3285'),
 Delayed('read_data-b9161ccc-186c-4cfd-9

Then we use [dask.dataframe.from_delayed](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.from_delayed). This function creates a Dask DataFrame from a list of delayed objects as long as each delayed object returns a pandas dataframe. The structure of each individual dataframe returned must also be the same.

In [24]:
df = dd.from_delayed(df, meta={'id': int, 'name': str, 'x': float, 'y': float})
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
                int64  object  float64  float64
                  ...     ...      ...      ...
...               ...     ...      ...      ...
                  ...     ...      ...      ...
                  ...     ...      ...      ...
Dask Name: from-delayed, 60 tasks

Note: we provided the dtypes in the `meta` keyword to explicitly tell Dask Dataframe what kind of dataframe to expect. If we did not do this Dask would infer this from the first delayed object which could be slow if it was a large csv file

## Now let's see if this works

In [25]:
df.head()

id    name         x         y
0  1057  Oliver -0.786602  0.821519
1  1037     Ray  0.739907 -0.630129
2  1002  Ursula -0.486140 -0.935097
3   994     Tim -0.701671 -0.958318
4   997   Frank  0.007467 -0.768376

In [26]:
df.describe().compute()

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


id             x             y
count  2.505600e+06  2.505600e+06  2.419200e+06
mean   1.000002e+03  4.984979e-05  4.070980e-04
std    3.159745e+01  5.773395e-01  5.770469e-01
min    8.230000e+02 -9.999994e-01 -9.999990e-01
25%    9.790000e+02 -4.932313e-01 -4.925213e-01
50%    1.000000e+03  9.024390e-03  7.174105e-03
75%    1.021000e+03  5.076600e-01  5.057808e-01
max    1.157000e+03  9.999992e-01  9.999981e-01

## Success!

To recap, in this example, we looked at an approach to create a Dask Dataframe from a collection of many data files. Typically you would use built-in functions like `dd.read_csv` or `dd.read_parquet` to do this. Sometimes, this is not possible because of messy/corrupted files in your dataset or some custom processing that might need to be done. 

In these cases, you can build a Dask DataFrame with the following steps.

1. Create a regular python function that reads the data, performs any transformations, error checking etc and always returns a  Pandas dataframe with the same structure
2. Convert this read function to a delayed object using the `dask.delayed` function
3. Call each file in your dataset with the delayed data reader and assemble the output as a list of delayed objects
4. Used `dd.from_delayed` to covert the list of delayed objects to a Dask Dataframe 

This same technique can be used in other situations as well. Another example might be data files that require using a specialized reader, or several transformations before they can be converted to a pandas dataframe.